# Cross Validation - Train  Validation  Test Split

Partea de Train-Test split are un dezavantaj mare și anume faptul că nu are o porțiune din setul de date care să fie cu adevărat 'neatinsă' și 'nevăzută' de către model pentru a putea face evaluare performanței unui anumit model. Deși partea de ajustare a hyperparametrilor utilizând setul de testare este extrem de folosită, aceasta totuși reprezintă o problemă în ceea ce privește partea de raportare a performanței modelului. Există o metodă prin care putem să prevenim această problemă.

Dacă dorim întradevăr să avem un model la care să nu facem îmbunătățire de hyperparametrii pe baza setului de testare (care ar trebuie să fie doar pentru partea de raport), există o calea și de a face acest lucru. Începem din nou cu setul de date întreg pe care îl avem. În loc să împărțim setul de date doar în train-test, acum mai adăugăm încă o categorie care poartă denumirea de valodation set. În acest moment o să avem trei noi categorii de seturi de date, train, validation și test. Procentajul în care distribuim setul de date depinde de mărimea setului de date inițial. Se dorește ca în setul de date de testare (test set) să fie un număr destul de reprezentativ de date pe care se poate testa. 

După ce avem aceste seturi de date, se pune deoparte setul de date de testare (test set), se antrenează un model pe setul de date de antrenare și se verifică performanța modelului (eroarea) pe setul de validare. Dacă nu suntem mulțumiți de performanța modelului atunci creem un nou model, îl reantrenăm pe setul de date de antrenare după care verificăm din nou performanaț modelului tot pe setul de date de verificare (validation set). În momentul în care sutem pe deplin mulțumiți de performanța modelului se mai rulează un nou set de predicții pe setul de date de testare și se evaluează modelul. După această evaluare nu mai avem voie să modificăm hyperparametrii modelului, eroare care rezultă pe setul de date de testare trebuie să fie cea care este raportată mai departe

Pentru a obține acest comportament în Python tot ce trebuie să facem este să rulăm metoda train_test_split() de două ori. După ce am rulat prima dată metoda și avem două seturi de date (train și test) mai rulăm încă odată metoda, însă de această dată doar pentru setul de test. Acest set îl împărțim acuma în validation set și test set. De setul de date 'test' nu trebuie să ne atingem până la final. Să implementăm toate acestea în Python și Scikit-Learn

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read the data into a DataFrame
df = pd.read_csv('../data/DATA/Advertising.csv')

In [3]:
# print the head of the data
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
# split the data into Features and Labels
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
# import train_test_split method from sklearn
from sklearn.model_selection import train_test_split

In [6]:
# split the data the first time into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

Acum că am apelat metoda train_test_split() prima dată avem două seturi de date, train (X_train, y_train) și test (X_test și y_test). Setul de date de testare o să îl mai împărțim încă odată utilizând din nou metoda train_test_split(). Diferența la aceasta a doua utilizare este faptul că nu o să utilizăm același procent de împărțire. O să împărțim setul de date de testare astfel încât să fie o rație de 50-50 (validation - test) sau 66-34 (validation - test)

In [7]:
# split the data the second time into validation-test sets
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=101)

În acest moment, după ce am rulat metoda de train_test_split() și a doua oară, acuma avem următoarele seturi de date:

1.  X_train (train set) = care reprezintă 70% din setul de date mare (df)

2.  X_validation (validation set) = care reprezintă 15% din setul de date mare (df)

3.  X_test (test set) = care reprezintă 15% din setul de date mare (df)

Pașii care trebuie făcuți în continare sunt următorii:

- crearea unui scaler

- antrenarea scaler-ului (pe datele de antrenare)

- trainsformarea datelor (pe toate cele trei seturi de date)

- crearea modelului

- evaluarea modelului (pe setul de date de validare)

- tunarea modelului

- chiar la final (după ce suntem mulțumiți cu performața modelului) trebuie evaluat modelul pe setul de date de testare

In [8]:
# mporting the StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

In [9]:
# creating an instance of the StandardScaler
scaler = StandardScaler()

In [10]:
# fitting the scaler only in train set
scaler.fit(X_train)

StandardScaler()

In [11]:
# transforming all the data with the fitted scaler
X_train = scaler.transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

In [12]:
# importing the Ridge model from Scikit-Learn
from sklearn.linear_model import Ridge

In [13]:
# creating an instance of the model
ridge1_model = Ridge(alpha=100)

In [14]:
# training the model on X_train (train set)
ridge1_model.fit(X_train, y_train)

Ridge(alpha=100)

După ce am creat și am atrenat un model de Ridge Regression, urmează partea de evaluare a modelului. Știm că inițial pentru partea de evaluare a unui model se făceau predicții pe un set de date, iar acele predicții erau comparate cu valorile adevărate de labels. Acest procedeu se făcea pentru setul de date de testare. Din moment ce am zis că partea de validare unui model pentru îmbunătățirea de hyperparametrii se va face utilizând setul de validare, acuma trebuie să folosim setul de validare pentru a face predicții și tot pentru acest set de validare o să calculăm erorile

In [15]:
y_validation_pred = ridge1_model.predict(X_validation)

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

MAE = mean_absolute_error(y_validation, y_validation_pred)
RMSE = np.sqrt(mean_squared_error(y_validation, y_validation_pred))

print(f'MAE = {MAE}')
print(f'RMSE = {RMSE}')

MAE = 2.1754243744399884
RMSE = 2.7055686017589484


După ce am verificat performanța modelului utilizând setul de date de validare, dacă nu suntem mulțumiți de rezltat putem să creem alt model cu alți hyperparametrii pentru a vedea dacă există ceva îmbunătățiri. O să creeem un nou model de Ridge Regression, de data aceasta cu valoarea 1 pentru parametrul de alpha și o să ăl evaluăm tot pe setul de date de validare

In [17]:
ridge2_model = Ridge(alpha=1)
ridge2_model.fit(X_train, y_train)

y_validation_pred = ridge2_model.predict(X_validation)

MAE = mean_absolute_error(y_validation, y_validation_pred)
RMSE = np.sqrt(mean_squared_error(y_validation, y_validation_pred))

print(f'MAE = {MAE}')
print(f'RMSE = {RMSE}')

MAE = 1.195143424023704
RMSE = 1.5439504768796786


După ce am modificat prametrii și suntem mulțumiți de rezultat putem să facem o nouă serie de predicții finale pentru setul de date de testare

In [18]:
y_pred = ridge2_model.predict(X_test)

MAE = mean_absolute_error(y_pred, y_test)
RMSE = np.sqrt(mean_squared_error(y_pred, y_test))

print(f'MAE = {MAE}')
print(f'RMSE = {RMSE}')

MAE = 1.2386102646924124
RMSE = 1.5014193564093001


Se poate vedea că performața modelului este diferită pentru validation set și test set. După ce am evaluat modelul pe test set nu mai avem voie să modificăm hyperparametrii modelului, perrformanța acestuia o să fie stailită ca fiind eroarea rezultată după evaluare modelului pe setul de date de testare.

## Recapitulare

În cadrul acestui tutorial am învățat următoarele lucruri:

    1. Existența unui validation set este necesară deoarece partea de verificare a performanței modelului și îmbunătățirea acestuia trebuie realizată după evaluarea modelului pe acest set de validare

    2. Cum să împărțim setul de date în seturile de antrenare, testare și validare

        X_train, X_validation_test, y_train, y_validation_test = train_test_split(X, y, test_size=0.3, random_state=42)

        X_validation, X_test, y_validation, y_test = train_test_split(X_validation_test, y_validation_test, test_size=0.5, random_state=42)

        